In [1]:
import os
import numpy as np
import pandas as pd

import tensorflow as tf
print(tf.__version__)

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2
2.5.0-rc2


### Basic architecture of a CNN

| Hyperparameter/Layer type | Description |
|:-------------------------:|-------------|
|   Input image(s)          | Images for discovering pattern |
|   Input layer             | Takes in target image and preprocess them for further layers     |
|   Convolution layer       | Extracts/Learns most importand patterns from images    |
|   Hidden activation       | Adds non-linearity to learned features       |
|   Pooling layer           | Reduces the dimensionality of learned image features        |
|   Fully connected layer   | Further refines learned features from convolution layers      |
|   Output layer            | Takes learned features and outputs them in shape of target labels       |
|   Output activation       | Adds non-linearity to output layer      |



_It's typically a stack of convolution layer and pooling layers_